In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
from sklearn.model_selection import train_test_split

In [2]:
#create custom data class

class CustomDataset(Dataset):
    def __init__(self, path):
        self.data = pd.read_csv(path, sep=",", header=1)
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sepal_length = torch.tensor(self.data.iloc[idx, 0])
        sepal_width = self.data.iloc[idx, 1]
        petal_length = self.data.iloc[idx, 2]
        petal_width = self.data.iloc[idx, 3] 
        label = self.data.iloc[idx, 4]
        
        tensor_data = torch.tensor([sepal_length,sepal_width,sepal_width,sepal_width],dtype=torch.float32)
        
        return tensor_data , label

In [3]:
#just my afwull way to seperate training and testing dataset
# 80% training , 20% Test

dataset = CustomDataset("./dataset/IRIS.csv")
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])
len(train_dataset),len(val_dataset)

(120, 29)

In [4]:
#create train_dataloader and test_dataloader
# 80% for training, 20% of data for testing -> 149*0.8 = 119.2 -> 120 , 29
batch_size = 10

train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True) 
test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [5]:
#4 inputs and 3  outputs 
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_stack =  nn.Sequential(
         nn.Linear(4,128),
         nn.ReLU(),
         nn.Linear(128,64),
         nn.ReLU(),
         nn.Linear(64,3),
         )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits

In [6]:
#map labels into tensor of shape [1, 0, 0]
def transform_label(label_data):
    data = []
    for i in label_data:
        if i == "Iris-setosa":
            #data.append(torch.tensor([1,0,0], dtype=torch.float32))
            data.append(torch.tensor(0))
        if i == "Iris-versicolor": 
            #data.append(torch.tensor([0,1,0], dtype=torch.float32))
            data.append(torch.tensor(1))
        if i == "Iris-virginica":
            #data.append(torch.tensor([0,0,1], dtype=torch.float32))
            data.append(torch.tensor(2))
    return torch.stack(data)

In [7]:
X_train, train_labels = next(iter(train_loader))
#train_labels before mapping tesnors 
train_labels

('Iris-virginica',
 'Iris-virginica',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-setosa',
 'Iris-setosa')

In [8]:
#train_labels, X_train = next(iter(train_loader))
train_labels = transform_label(train_labels)
X_train.shape, train_labels.shape, X_train.dtype, train_labels.dtype
train_labels

tensor([2, 2, 0, 1, 2, 0, 1, 2, 0, 0])

In [9]:
X_train, train_labels

(tensor([[7.2000, 3.2000, 3.2000, 3.2000],
         [6.9000, 3.1000, 3.1000, 3.1000],
         [5.1000, 3.8000, 3.8000, 3.8000],
         [6.3000, 2.3000, 2.3000, 2.3000],
         [6.7000, 3.0000, 3.0000, 3.0000],
         [4.7000, 3.2000, 3.2000, 3.2000],
         [5.0000, 2.3000, 2.3000, 2.3000],
         [6.4000, 3.2000, 3.2000, 3.2000],
         [5.1000, 3.8000, 3.8000, 3.8000],
         [5.2000, 3.4000, 3.4000, 3.4000]]),
 tensor([2, 2, 0, 1, 2, 0, 1, 2, 0, 0]))

In [12]:
#create model
lr = 0.01
model = NeuralNetwork()
optim = torch.optim.SGD(model.parameters(), lr=lr)
loss = torch.nn.CrossEntropyLoss()
#loss = torch.nn.MSELoss()
#Weights are by default torch.32 not 64 --> error message 

In [13]:
n_iters = 500
steps = n_iters/10
LOSS = []
for epochs in range(n_iters):  
    for i,(inputs, labels) in enumerate(train_loader):
        out = model(inputs)
        train_labels = transform_label(labels)
        l = loss(out, train_labels)
        l.backward()
        #update weights
        optim.step()
        optim.zero_grad()
    LOSS.append(l.item())
    if epochs%steps == 0:
        print(f"\n epoch: {int(epochs+steps)}/{n_iters}, loss: {sum(LOSS)/len(LOSS)}")
        #if i % 1 == 0:
            #print(f" steps: {i+1}, loss : {l.item()}")


 epoch: 50/500, loss: 1.0882762670516968

 epoch: 100/500, loss: 0.6235557990331276

 epoch: 150/500, loss: 0.5674979658705173

 epoch: 200/500, loss: 0.5400670744133311

 epoch: 250/500, loss: 0.5218866834444786

 epoch: 300/500, loss: 0.513644625347449

 epoch: 350/500, loss: 0.5026437971382046

 epoch: 400/500, loss: 0.4981175306176188

 epoch: 450/500, loss: 0.49155918209630056

 epoch: 500/500, loss: 0.48949935473914685


In [ ]:
#X_test, test_labels = next(iter(test_loader))
#test_labels = transform_label(test_labels)
#x_label_predicted = model(X_test)
#x_label_predicted, test_labels

In [ ]:
#""""
#def final_x(x_hat):
#    list_tensors = []
#    for i in x_hat:
#        print(i)
#        final_tensor = torch.zeros(1,3)
#        final_tensor[0:,i] = 1
#        list_tensors.append(final_tensor)
#    return torch.cat(list_tensors)
#    
#    # tensor[0:,1] = 4 to accses specific value 
#_, x_label_predicted_hat = torch.max(x_label_predicted,1)
#x_label_predicted_hat

In [ ]:
#final_x = final_x(x_label_predicted_hat)

In [ ]:
def get_accuracy(model_prediction, test_labels):
    idx = 0
    number_pred = 0
    counter = 0
    while idx < 10:
        print(f" \n Tensor[{idx+1}]")
        for i in range(3):
            print(f"model: {model_prediction[idx][i].item()}, label: {test_labels[idx][i].item()}")
            if model_prediction[idx][i].item() == test_labels[idx][i].item():

                counter += 1
                if counter == 3:
                    number_pred += 1
        counter = 0
          
        idx +=1
    accuracy = number_pred/10 *100
    return print(f"\naccuracy of model {number_pred/10 *100}%")

In [ ]:
#get_accuracy(final_x, test_labels)

In [ ]:
#test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)
import accuracy_model as ac
test_model_acc = ac.data_loader
test_model_acc.accuracy(model, test_loader, batch_size=10)

In [ ]:
# save model 
#torch.save(model.state_dict(), "./model")